In [3]:
import pandas as pd
import numpy as np
import plotly.express as px
import scipy.stats as stats
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [4]:
df = pd.read_excel('../../data/globalterrorismdb_0522dist.xlsx')


In [5]:
pd.set_option('display.max_columns', None)
df.head()

,eventid,iyear,imonth,iday,approxdate,extended,resolution,country,country_txt,region,region_txt,provstate,city,latitude,longitude,specificity,vicinity,location,summary,crit1,crit2,crit3,doubtterr,alternative,alternative_txt,multiple,success,suicide,attacktype1,attacktype1_txt,attacktype2,attacktype2_txt,attacktype3,attacktype3_txt,targtype1,targtype1_txt,targsubtype1,targsubtype1_txt,corp1,target1,natlty1,natlty1_txt,targtype2,targtype2_txt,targsubtype2,targsubtype2_txt,corp2,target2,natlty2,natlty2_txt,targtype3,targtype3_txt,targsubtype3,targsubtype3_txt,corp3,target3,natlty3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,motive,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weaptype1,weaptype1_txt,weapsubtype1,weapsubtype1_txt,weaptype2,weaptype2_txt,weapsubtype2,weapsubtype2_txt,weaptype3,weaptype3_txt,weapsubtype3,weapsubtype3_txt,weaptype4,weaptype4_txt,weapsubtype4,weapsubtype4_txt,weapdetail,nkill,nkillus,nkillter,nwound,nwoundus,nwoundte,property,propextent,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhostkidus,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome,hostkidoutcome_txt,nreleased,addnotes,scite1,scite2,scite3,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related
0,197000000001,1970,7,2,NaN,0,NaT,58,Dominican Republic,2,Central America & Caribbean,National,Santo Domingo,18.456792,-69.951164,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,14,Private Citizens & Property,68.0,Named Civilian,NaN,Julio Guzman,58.0,Dominican Republic,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MANO-D,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,0,0,0,NaN
1,197000000002,1970,0,0,NaN,0,NaT,130,Mexico,1,North America,Federal,Mexico city,19.371887,-99.086624,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,6,Hostage Taking (Kidnapping),NaN,NaN,NaN,NaN,7,Government (Diplomatic),45.0,"Diplomatic Personnel (outside of embassy, cons...",Belgian Ambassador Daughter,"Nadine Chaval, daughter",21.0,Belgium,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23rd of September Communist League,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,7.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,1.0,1.0,0.0,NaN,NaN,NaN,Mexico,1.0,800000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,0,1,1,1,NaN
2,197001000001,1970,1,0,NaN,0,NaT,160,Philippines,5,Southeast Asia,Tarlac,Unknown,15.478598,120.599741,4.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,1,Assassination,NaN,NaN,NaN,NaN,10,Journalists & Media,54.0,Radio Journalist/Staff/Facility,Voice of America,Employee,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,PGIS,-9,-9,1,1,NaN
3,197001000002,1970,1,0,NaN,0,NaT,78,Greece,8,Western Europe,Attica,Athens,37.997490,23.762728,1.0,0,NaN,NaN,1,1,1,0,NaN,NaN,0.0,1,0,3,Bombing/Explosion,NaN,NaN,NaN,NaN,7,Government (Diplomatic),46.0,Embassy/Consulate,NaN,U.S. Embassy,217.0,United States,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,Explosives,16.0,Unknown Explosive Type,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Explosive,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,Na

In [6]:
def get_boolean_like_columns(df):
    boolean_columns = []
    boolean_columns.extend(df.select_dtypes(include=['bool']).columns)
    for col in df.select_dtypes(include=['number']).columns:
        unique_values = df[col].dropna().unique()
        if set(unique_values).issubset({0, 1, 0.0, 1.0, False, True}):
            boolean_columns.append(col)
    
    return boolean_columns

# Récupération des colonnes booléennes
boolean_columns = get_boolean_like_columns(df)
print("Colonnes booléennes probables :", boolean_columns)

Colonnes booléennes probables : ['extended', 'crit1', 'crit2', 'crit3', 'multiple', 'success', 'suicide', 'guncertain1', 'guncertain2', 'guncertain3', 'individual', 'claim3']


In [7]:
from mlxtend.frequent_patterns import apriori, association_rules
df_bool = df[boolean_columns]
df_bool.head()

# remplacement des NaN par 0
df_bool.fillna(0, inplace=True)
supports = apriori(df_bool, min_support=0.1, use_colnames=True)

# Calculer les règles d'association (optionnel)
rules = association_rules(supports, metric="leverage", min_threshold=1)

supports.sort_values(by="support", ascending=False).head(10)



C:\Users\emili\AppData\Local\Temp\ipykernel_11644\789458553.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bool.fillna(0, inplace=True)
c:\Users\benva\.conda\envs\cuda\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
1,0.993291,(crit2)
0,0.988265,(crit1)
5,0.981555,"(crit2, crit1)"
4,0.883628,(success)
11,0.877338,"(crit2, success)"
8,0.872893,"(success, crit1)"
17,0.866604,"(success, crit2, crit1)"
2,0.866246,(crit3)
9,0.859537,"(crit2, crit3)"
6,0.854511,"(crit3, crit1)"


In [8]:
supports["len"] = supports["itemsets"].apply(lambda x: len(x))
pd.set_option("display.max_colwidth", None)
supports[supports["len"]==2].sort_values(by="support", ascending=False).head(10)

,support,itemsets,len
5,0.981555,"(crit2, crit1)",2
11,0.877338,"(crit2, success)",2
8,0.872893,"(success, crit1)",2
9,0.859537,"(crit2, crit3)",2
6,0.854511,"(crit3, crit1)",2
13,0.765972,"(success, crit3)",2
7,0.145380,"(crit1, multiple)",2
10,0.145241,"(crit2, multiple)",2
14,0.131041,"(success, multiple)",2
12,0.130597,"(multiple, crit3)",2


In [9]:
# compute association rules
rules = association_rules(supports, metric="confidence", min_threshold=0.5)
rules.head(50)

# filtrer par leverage descroissant
filtered_rules = rules[
    (rules["support"] >= 0.05) &
    (rules["confidence"] >= 0.7) &
    (rules["lift"] > 1) &
    (rules["leverage"] > 0) &
    (rules["conviction"] > 1.2)
]


filtered_rules = filtered_rules.sort_values(by="lift", ascending=False)

filtered_rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
114,(multiple),"(crit2, crit1, crit3, success)",0.145899,0.748949,0.117708,0.806772,1.077206,0.008436,1.299251,0.083916
92,(multiple),"(crit1, crit3, success)",0.145899,0.755238,0.118351,0.811184,1.074078,0.008163,1.296301,0.080750
110,"(crit2, multiple)","(crit1, crit3, success)",0.145241,0.755238,0.117708,0.810427,1.073075,0.008016,1.291124,0.079670
99,(multiple),"(crit2, crit3, success)",0.145899,0.759683,0.118175,0.809975,1.066202,0.007338,1.264663,0.072698
112,"(crit1, multiple)","(crit2, crit3, success)",0.145380,0.759683,0.117708,0.809657,1.065783,0.007265,1.262546,0.072222
57,(multiple),"(crit3, success)",0.145899,0.765972,0.118819,0.814388,1.063207,0.007064,1.260840,0.069605
91,"(crit1, multiple)","(crit3, success)",0.145380,0.765972,0.118351,0.814085,1.062812,0.006995,1.258786,0.069153
96,"(crit2, multiple)","(crit3, success)",0.145241,0.765972,0.118175,0.813645,1.062238,0.006924,1.255816,0.068547
105,"(crit2, crit1, multiple)","(crit3, success)",0.144722,0.765972,0.117708,0.813338,1.061837,0.006855,1.253752,0.068090
111,"(success, multiple)","(crit2, crit3, crit1)",0.131041,0.847801,0.117708,0.898253,1.059509,0.006611,1.495859,0.064637


From the observation we made, we can see that lift is around 1.0 that means the items are independent of each other. We will try to to use our preprocessed data to see if we can get pattern frequent itemsets.

In [10]:
df2 = pd.read_csv('../../data/processed/db_2021_preprocessed.csv')
df2.head()

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targsubtype1_txt,corp1,natlty1_txt,targsubtype2_txt,corp2,natlty2_txt,targsubtype3_txt,corp3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication
0,202101010004,2021-01-01,2021-01-01,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1,0,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Military Transportation/Vehicle (excluding convoys),Egyptian Armed Forces,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remote Trigger,NaN,NaN,NaN,NaN,2.0,0.0,5.0,0.0,Unknown,NaN,Vehicle damaged,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,202101010005,2021-01-01,2021-01-01,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1,0,1,1,1,0,NaN,0.0,1,0,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),India,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenade,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,202101010006,2021-01-01,2021-01-01,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1,1,1,1,1,1,State Actors,0.0,1,0,"Procession/Gathering (funeral, wedding, birthday, religious)",Almansour Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,Minor (likely < $1 million),NaN,Building damaged,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,0,0,0,0,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,202101010009,2021-01-01,2021-01-01,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1,1,1,1,1,1,State Actors,0.0,1,0,Radio Journalist/Staff/Facility,Sadai Ghor Radio,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Gun Type,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,Minor (likely < $1 million),NaN,Vehicle damaged,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,

In [11]:

# Récupération des colonnes booléennes
boolean_columns = get_boolean_like_columns(df2)
print("Colonnes booléennes probables :", boolean_columns)

Colonnes booléennes probables : ['vicinity', 'crit1', 'crit2', 'crit3', 'doubtterr', 'multiple', 'success', 'suicide', 'gsubname3', 'guncertain1', 'guncertain2', 'guncertain3', 'individual', 'claimed', 'claim2', 'claim3', 'compclaim', 'weapsubtype4_txt', 'INT_MISC', 'weapon_biological', 'weapon_chemical', 'weapon_radiological', 'weapon_nuclear', 'weapon_firearms', 'weapon_explosives', 'weapon_fake weapons', 'weapon_incendiary', 'weapon_melee', 'weapon_vehicle', 'weapon_sabotage equipment', 'weapon_other', 'weapon_unknown', 'attack_assassination', 'attack_armed assault', 'attack_bombing/explosion', 'attack_hijacking', 'attack_hostage taking (barricade incident)', 'attack_hostage taking (kidnapping)', 'attack_facility/infrastructure attack', 'attack_unarmed assault', 'attack_unknown', 'target_business', 'target_government (general)', 'target_police', 'target_military', 'target_abortion related', 'target_airports & aircraft', 'target_government (diplomatic)', 'target_educational instituti

In [12]:
from mlxtend.frequent_patterns import apriori, association_rules
df_bool = df2[boolean_columns]

# remplacement des NaN par 0
df_bool.fillna(0, inplace=True)

df_bool.head()

# drop columns with unknown in the name like weapon_unknown
df_bool = df_bool[df_bool.columns.drop(list(df_bool.filter(regex='_unknown')))]


C:\Users\emili\AppData\Local\Temp\ipykernel_11644\578340805.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bool.fillna(0, inplace=True)


In [13]:
supports = apriori(df_bool, min_support=0.05, use_colnames=True)


supports.sort_values(by="support", ascending=False).head(10)


c:\Users\emili\anaconda3\envs\dm\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
2,0.996976,(crit2)
1,0.985081,(crit1)
25,0.982056,"(crit2, crit1)"
6,0.851613,(success)
48,0.848992,"(crit2, success)"
29,0.838911,"(success, crit1)"
144,0.836290,"(success, crit2, crit1)"
3,0.785484,(crit3)
45,0.782460,"(crit2, crit3)"
26,0.770565,"(crit3, crit1)"


In [14]:
supports["len"] = supports["itemsets"].apply(lambda x: len(x))
pd.set_option("display.max_colwidth", None)
supports[supports["len"]==2].sort_values(by="support", ascending=False).head(10)

,support,itemsets,len
25,0.982056,"(crit2, crit1)",2
48,0.848992,"(crit2, success)",2
29,0.838911,"(success, crit1)",2
45,0.782460,"(crit2, crit3)",2
26,0.770565,"(crit3, crit1)",2
65,0.665927,"(crit3, success)",2
52,0.420565,"(crit2, weapon_explosives)",2
33,0.418548,"(weapon_explosives, crit1)",2
51,0.384879,"(crit2, weapon_firearms)",2
32,0.375806,"(crit1, weapon_firearms)",2


In [15]:
# compute association rules
rules = association_rules(supports, metric="confidence", min_threshold=0.5)
rules.head(50)

# filtrer par leverage descroissant
filtered_rules = rules[
    (rules["support"] >= 0.05) &
    (rules["confidence"] >= 0.7) &
    (rules["lift"] > 1) &
    (rules["leverage"] > 0) &
    (rules["conviction"] > 1.2)
]


filtered_rules = filtered_rules.sort_values(by="leverage", ascending=False)

filtered_rules.head(50)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
92,(weapon_explosives),(attack_bombing/explosion),0.421169,0.370161,0.370161,0.878889,2.374342,0.214261,5.200526,1.000000
91,(attack_bombing/explosion),(weapon_explosives),0.370161,0.421169,0.370161,1.000000,2.374342,0.214261,inf,0.919014
485,(attack_bombing/explosion),"(crit2, weapon_explosives)",0.370161,0.420565,0.369758,0.998911,2.375166,0.214081,531.921774,0.919246
484,"(crit2, weapon_explosives)",(attack_bombing/explosion),0.420565,0.370161,0.369758,0.879195,2.375166,0.214081,5.213665,0.999208
483,"(attack_bombing/explosion, crit2)",(weapon_explosives),0.369758,0.421169,0.369758,1.000000,2.374342,0.214027,inf,0.918426
486,(weapon_explosives),"(attack_bombing/explosion, crit2)",0.421169,0.369758,0.369758,0.877932,2.374342,0.214027,5.163041,1.000000
315,"(weapon_explosives, crit1)",(attack_bombing/explosion),0.418548,0.370161,0.368145,0.879576,2.376197,0.213215,5.230181,0.996058
316,(attack_bombing/explosion),"(weapon_explosives, crit1)",0.370161,0.418548,0.368145,0.994553,2.376197,0.213215,106.754516,0.919536
314,"(attack_bombing/explosion, crit1)",(weapon_explosives),0.368145,0.421169,0.368145,1.000000,2.374342,0.213094,inf,0.916082
317,(weapon_explosives),"(attack_bombing/explosion, crit1)",0.421169,0.368145,0.368145,0.874102,2.374342,0.213094,5.018801,1.000000


# Analysis patterns
- Rule 5392
    - Antecedents : (doubtterr, crit2, crit1, attack_armed assault, success)
    - Consequents : (weapon_firearms, target_military)

- Support: 5.99%, a moderate proportion.
- Confidence: 94.29%, indicating that almost all incidents meeting these criteria involve firearms and target a military target.
- Lift: 10.98, a very strong relationship.
- Context: This suggests that these criteria (crit1, crit2, doubtterr) are strong indicators of armed incidents targeting military targets.
------------
- Rule 5434
    - Antecedents : (success, attack_armed assault, target_military)
    - Consequents : (weapon_firearms, doubtterr, crit2, crit1)

- Support: 5.99%, same as rule 5392.
- Confidence: 79.62%, slightly lower than the previous rule.
- Lift: 10.85, still very high.
- Context: Successful incidents against military targets in armed attacks are strongly associated with the use of firearms and the mentioned criteria.
-------------
- Rule 4568
    - Antecedents : (crit1, doubtterr, attack_armed assault, crit2)
    - Consequents : (weapon_firearms, target_military)

- Support: 6.69%, slightly more frequent than other rules.
- Confidence: 92.73%, very high, with a lift of 10.79.
- Lift: 10.80 : high
- Context: These criteria (including crit1, crit2, doubtterr) increase the likelihood that firearms will be used against military targets.
-------------

The criteria crit1, crit2, and doubtterr play a central role in defining armed incidents against military targets.

Successful incidents involving firearms have a very high probability of involving military targets.

The very strong relationships (high lift, high conviction) suggest that these patterns are significant in the context of terrorism.


In [22]:
df_looking_group_type= pd.read_csv('../../data/processed/db_2021_preprocessed.csv')
df_looking_group_type.head()

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targsubtype1_txt,corp1,natlty1_txt,targsubtype2_txt,corp2,natlty2_txt,targsubtype3_txt,corp3,natlty3_txt,gname,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication
0,202101010004,2021-01-01,2021-01-01,Egypt,Middle East & North Africa,North Sinai,Beir al-Abd,31.016779,33.008587,1,0,1,1,0,1,Insurgency/Guerilla Action,0.0,1,0,Military Transportation/Vehicle (excluding convoys),Egyptian Armed Forces,Egypt,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Remote Trigger,NaN,NaN,NaN,NaN,2.0,0.0,5.0,0.0,Unknown,NaN,Vehicle damaged,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,202101010005,2021-01-01,2021-01-01,India,South Asia,Jammu and Kashmir,Pampore,34.010230,74.917261,1,0,1,1,1,0,NaN,0.0,1,0,"Police Building (headquarters, station, school)",Central Reserve Police Force (CRPF),India,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Grenade,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,0.0,NaN,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,202101010006,2021-01-01,2021-01-01,Yemen,Middle East & North Africa,Al Hudaydah,Hudaydah,14.782391,42.973093,1,1,1,1,1,1,State Actors,0.0,1,0,"Procession/Gathering (funeral, wedding, birthday, religious)",Almansour Wedding Hall,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,Houthi extremists (Ansar Allah),NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Projectile (rockets, mortars, RPGs, etc.)",NaN,NaN,NaN,NaN,5.0,0.0,7.0,0.0,Minor (likely < $1 million),NaN,Building damaged,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,0,0,0,0,NaN,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,202101010009,2021-01-01,2021-01-01,Afghanistan,South Asia,Ghor,Feroz Koh,34.521315,65.249242,1,1,1,1,1,1,State Actors,0.0,1,0,Radio Journalist/Staff/Facility,Sadai Ghor Radio,Afghanistan,NaN,NaN,NaN,NaN,NaN,NaN,Unknown,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,0,-99,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Unknown Gun Type,NaN,NaN,NaN,NaN,1.0,0.0,0.0,0.0,Minor (likely < $1 million),NaN,Vehicle damaged,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,START Primary Collection,-9,-9,0,-9,NaN,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,

In [23]:
# One hot encode gname

df_looking_group_type = pd.get_dummies(df_looking_group_type, columns=['gname'])
df_looking_group_type.head()

,eventid,begin_date,end_date,country_txt,region_txt,provstate,city,latitude,longitude,specificity,vicinity,crit1,crit2,crit3,doubtterr,alternative_txt,multiple,success,suicide,targsubtype1_txt,corp1,natlty1_txt,targsubtype2_txt,corp2,natlty2_txt,targsubtype3_txt,corp3,natlty3_txt,gsubname,gname2,gsubname2,gname3,gsubname3,guncertain1,guncertain2,guncertain3,individual,nperps,nperpcap,claimed,claimmode_txt,claim2,claimmode2,claimmode2_txt,claim3,claimmode3,claimmode3_txt,compclaim,weapsubtype1_txt,weapsubtype2,weapsubtype2_txt,weapsubtype3_txt,weapsubtype4_txt,nkill,nkillter,nwound,nwoundte,propextent_txt,propvalue,propcomment,ishostkid,nhostkid,nhours,ndays,divert,kidhijcountry,ransom,ransomamt,ransomamtus,ransompaid,ransompaidus,ransomnote,hostkidoutcome_txt,nreleased,dbsource,INT_LOG,INT_IDEO,INT_MISC,INT_ANY,related,weapon_biological,weapon_chemical,weapon_radiological,weapon_nuclear,weapon_firearms,weapon_explosives,weapon_fake weapons,weapon_incendiary,weapon_melee,weapon_vehicle,weapon_sabotage equipment,weapon_other,weapon_unknown,attack_assassination,attack_armed assault,attack_bombing/explosion,attack_hijacking,attack_hostage taking (barricade incident),attack_hostage taking (kidnapping),attack_facility/infrastructure attack,attack_unarmed assault,attack_unknown,target_business,target_government (general),target_police,target_military,target_abortion related,target_airports & aircraft,target_government (diplomatic),target_educational institution,target_food or water supply,target_journalists & media,target_maritime,target_ngo,target_other,target_private citizens & property,target_religious figures/institutions,target_telecommunication,gname_Abu Sayyaf Group (ASG),gname_Afrin Liberation Forces (HRE),gname_Ahl al-Marouf,gname_Al-Aqsa Martyrs Brigade,gname_Al-Badr,gname_Al-Islah Party,gname_Al-Khobar,gname_Al-Qaida in the Arabian Peninsula (AQAP),gname_Al-Qaida in the Indian Subcontinent,gname_Al-Qaida in the Islamic Maghreb (AQIM),gname_Al-Shabaab,gname_Alipour militia,gname_All Progressives Congress (APC),gname_Alliance of Patriots for a Free and Sovereign Congo (APCLS),gname_Allied Democratic Forces (ADF),gname_Alwiya Al-Waad Al-Haq (Truthful Promise Brigades),gname_Ambazonia Restoration Forces (ARF),gname_Anarchists,gname_Ansar Abi Bakr al-Siddiq Squadron,gname_Ansar Allah al-Awfiya,gname_Ansar al-Sunna (Mozambique),gname_Anti-Asian extremists,gname_Anti-Balaka Militia,gname_Anti-Coup extremists,gname_Anti-Government extremists,gname_Anti-Immigrant extremists,gname_Anti-Kurdish extremists,gname_Anti-Muslim extremists,gname_Anti-Police extremists,gname_Anti-Semitic extremists,gname_Arab-Israeli extremists,gname_Arakan Army (AA),gname_Arakan Liberation Party (ALP),gname_Arauco Malleco Coordinating Group (CAM) - Chile,gname_Armed Peasant Association (ACA-EP),gname_Asa'ib Ahl al-Haqq,gname_Ashab al-Kahf,gname_Atanga Nji Militia,gname_Baloch Liberation Army (BLA),gname_Baloch Liberation Front (BLF),gname_Baloch Republican Army (BRA),gname_Baloch Republican Guards (BRG),gname_Bangsamoro Islamic Freedom Movement (BIFM),gname_Boko Haram,gname_Celulas Revolucionarias Nicolas Neira,gname_Central Africa Province of the Islamic State,gname_Chinland Defense Force,gname_Coalition of Patriots for Change (CPC),gname_Committee for the Abolition of 5G and Its World (CLA5GSM),gname_Communist Party of India - Maoist (CPI-Maoist),gname_Communist Party of Nepal - Maoist (CPN-Maoist-Chand),gname_Conspiracy theory extremists,gname_Continuity Irish Republican Army (CIRA),gname_Cooperative for the Development of the Congo (CODECO) militia,gname_Dan Na Ambassagou,gname_Democratic Front for the Liberation of Rwanda (FDLR),gname_Democratic Union Party (PYD),gname_Dimasa National Liberation Army (DNLA),gname_Dissident Republicans,gname_Donetsk People's Republic,gname_East Asia Division of the Islamic State,gname_Fako Action Restoration Forces,gname_Fezzan Province of the Islamic State,gname_Free Papua Movement (OPM-Organisasi Papua Merdeka),gname

In [24]:
df_looking_group_type_boolean_col = get_boolean_like_columns(df_looking_group_type)
print("Colonnes booléennes probables :", df_looking_group_type_boolean_col)
df_looking_group_type_boolean = df_looking_group_type[df_looking_group_type_boolean_col]
df_looking_group_type_boolean.head()

# columns_to_drop = [
#     'vicinity', 'crit1', 'crit2', 'crit3', 'doubtterr', 'multiple', 
#     'success', 'suicide', 'gsubname3', 'guncertain1', 'guncertain2', 
#     'guncertain3', 'individual', 'claimed', 'claim2', 'claim3', 
#     'compclaim', 'weapsubtype4_txt', 'INT_MISC', 'weapon_biological', 
#     'weapon_chemical', 'weapon_radiological', 'weapon_nuclear', 
#     'weapon_firearms', 'weapon_explosives', 'weapon_fake weapons', 
#     'weapon_incendiary', 'weapon_melee', 'weapon_vehicle', 
#     'weapon_sabotage equipment', 'weapon_other', 'weapon_unknown', 
#     'target_business', 'target_government (general)', 'target_police', 
#     'target_military', 'target_abortion related', 'target_airports & aircraft', 
#     'target_government (diplomatic)', 'target_educational institution', 
#     'target_food or water supply', 'target_journalists & media', 
#     'target_maritime', 'target_ngo', 'target_other', 
#     'target_private citizens & property', 'target_religious figures/institutions', 
#     'target_telecommunication'
# ]

# # Dropping the columns from the DataFrame
# df_looking_group_type_boolean = df_looking_group_type_boolean.drop(columns=columns_to_drop)


Colonnes booléennes probables : ['gname_Abu Sayyaf Group (ASG)', 'gname_Afrin Liberation Forces (HRE)', 'gname_Ahl al-Marouf', 'gname_Al-Aqsa Martyrs Brigade', 'gname_Al-Badr', 'gname_Al-Islah Party', 'gname_Al-Khobar', 'gname_Al-Qaida in the Arabian Peninsula (AQAP)', 'gname_Al-Qaida in the Indian Subcontinent', 'gname_Al-Qaida in the Islamic Maghreb (AQIM)', 'gname_Al-Shabaab', 'gname_Alipour militia', 'gname_All Progressives Congress (APC)', 'gname_Alliance of Patriots for a Free and Sovereign Congo (APCLS)', 'gname_Allied Democratic Forces (ADF)', 'gname_Alwiya Al-Waad Al-Haq (Truthful Promise Brigades)', 'gname_Ambazonia Restoration Forces (ARF)', 'gname_Anarchists', 'gname_Ansar Abi Bakr al-Siddiq Squadron', 'gname_Ansar Allah al-Awfiya', 'gname_Ansar al-Sunna (Mozambique)', 'gname_Anti-Asian extremists', 'gname_Anti-Balaka Militia', 'gname_Anti-Coup extremists', 'gname_Anti-Government extremists', 'gname_Anti-Immigrant extremists', 'gname_Anti-Kurdish extremists', 'gname_Anti-Mu

,gname_Abu Sayyaf Group (ASG),gname_Afrin Liberation Forces (HRE),gname_Ahl al-Marouf,gname_Al-Aqsa Martyrs Brigade,gname_Al-Badr,gname_Al-Islah Party,gname_Al-Khobar,gname_Al-Qaida in the Arabian Peninsula (AQAP),gname_Al-Qaida in the Indian Subcontinent,gname_Al-Qaida in the Islamic Maghreb (AQIM),gname_Al-Shabaab,gname_Alipour militia,gname_All Progressives Congress (APC),gname_Alliance of Patriots for a Free and Sovereign Congo (APCLS),gname_Allied Democratic Forces (ADF),gname_Alwiya Al-Waad Al-Haq (Truthful Promise Brigades),gname_Ambazonia Restoration Forces (ARF),gname_Anarchists,gname_Ansar Abi Bakr al-Siddiq Squadron,gname_Ansar Allah al-Awfiya,gname_Ansar al-Sunna (Mozambique),gname_Anti-Asian extremists,gname_Anti-Balaka Militia,gname_Anti-Coup extremists,gname_Anti-Government extremists,gname_Anti-Immigrant extremists,gname_Anti-Kurdish extremists,gname_Anti-Muslim extremists,gname_Anti-Police extremists,gname_Anti-Semitic extremists,gname_Arab-Israeli extremists,gname_Arakan Army (AA),gname_Arakan Liberation Party (ALP),gname_Arauco Malleco Coordinating Group (CAM) - Chile,gname_Armed Peasant Association (ACA-EP),gname_Asa'ib Ahl al-Haqq,gname_Ashab al-Kahf,gname_Atanga Nji Militia,gname_Baloch Liberation Army (BLA),gname_Baloch Liberation Front (BLF),gname_Baloch Republican Army (BRA),gname_Baloch Republican Guards (BRG),gname_Bangsamoro Islamic Freedom Movement (BIFM),gname_Boko Haram,gname_Celulas Revolucionarias Nicolas Neira,gname_Central Africa Province of the Islamic State,gname_Chinland Defense Force,gname_Coalition of Patriots for Change (CPC),gname_Committee for the Abolition of 5G and Its World (CLA5GSM),gname_Communist Party of India - Maoist (CPI-Maoist),gname_Communist Party of Nepal - Maoist (CPN-Maoist-Chand),gname_Conspiracy theory extremists,gname_Continuity Irish Republican Army (CIRA),gname_Cooperative for the Development of the Congo (CODECO) militia,gname_Dan Na Ambassagou,gname_Democratic Front for the Liberation of Rwanda (FDLR),gname_Democratic Union Party (PYD),gname_Dimasa National Liberation Army (DNLA),gname_Dissident Republicans,gname_Donetsk People's Republic,gname_East Asia Division of the Islamic State,gname_Fako Action Restoration Forces,gname_Fezzan Province of the Islamic State,gname_Free Papua Movement (OPM-Organisasi Papua Merdeka),gname_Free Syrian Army,gname_Front for Change and Concord in Chad (FACT),gname_Front for the Restoration of Unity and Democracy,gname_Fulani extremists,gname_Grey Wolves,gname_Gumuz militia,gname_Hafiz Gul Bahadur Group,gname_Haftar Militia,gname_Hamas (Islamic Resistance Movement),gname_Hay'at Tahrir al-Sham,gname_Hezbollah,gname_Hind Province of the Islamic State,gname_Hizbul Mujahideen (HM),gname_Honneur et Nation,gname_Houthi extremists (Ansar Allah),gname_Hsipaw Civilian Protection Group,gname_Hurras al-Din,gname_Indigenous People of Biafra (IPOB),gname_International Resistance,gname_Iran-backed militia,gname_Islamic State in the Greater Sahara (ISGS),gname_Islamic State of Iraq and the Levant (ISIL),gname_Israeli settlers,gname_Jabha East Africa,gname_Jaish al-Adl,gname_Jaish-e-Mohammad (JeM),gname_Jama'atul Mujahideen Bangladesh (JMB),gname_Jamaah Ansharut Daulah,gname_Jamaat Nusrat al-Islam wal Muslimin (JNIM),gname_Janatantrik Terai Mukti Morcha- Goit (JTMM-G),gname_Janjaweed,gname_Jharkhand Janmukti Parishad (JJP),gname_Jihadi-inspired extremists,gname_Jund al-Khilafah (Tunisia),gname_Kachin Independence Army (KIA),gname_Kalay Region Defense Association (KRDA),gname_Karen National Union,gname_Kata'ib Hezbollah,gname_Khorasan Chapter of the Islamic State,gname_Khyber Brigade,gname_Kuki National Liberation Front (KNLF),gname_Kurdistan Free Life Party,gname_Kurdistan Workers' Party (PKK),gname_Lashkar-e-Mustafa (LeM),gname_Lashkar-e-Taiba (LeT),gname_Left-wing extremists,gname_Lions of Khilafah in the Maldives,gname_Lord's Resistance Army (LRA),gname_Luhansk People's Republic,gname_Mai Mai Bakata Katanga Militia,gname_Mai Mai Kyandenga,gname

In [25]:
df_looking_group_type_boolean.fillna(0, inplace=True)

# df_looking_group_type_boolean = df_looking_group_type_boolean[df_looking_group_type_boolean.columns.drop(list(df_looking_group_type_boolean.filter(regex='_unknown')))]
# df_looking_group_type_boolean = df_looking_group_type_boolean[df_looking_group_type_boolean.columns.drop(list(df_looking_group_type_boolean.filter(regex='Unknown')))]

# print all columns
pd.set_option('display.max_columns', None)
df_looking_group_type_boolean.head()

supports = apriori(df_looking_group_type_boolean, min_support=0.005, use_colnames=True)

# rules = association_rules(supports, metric="lift", min_threshold=1.0)

# rules.head(50)

C:\Users\emili\AppData\Local\Temp\ipykernel_11644\1898800230.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_looking_group_type_boolean.fillna(0, inplace=True)
c:\Users\emili\anaconda3\envs\dm\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [ ]:
rules = association_rules(supports, metric="lift", min_threshold=0.05)
rules.head(50)
gname_pattern = 'gname'
attack_pattern = 'attack'
# # filtrer par leverage descroissant
# filtered_rules = rules[
#     (rules["support"] >= 0.05) &
#     (rules["confidence"] >= 0.7) &
#     (rules["lift"] > 1) &
#     (rules["leverage"] > 0) &
#     (rules["conviction"] > 1.2)
# ]

filtered_rules = rules

filtered_rules = filtered_rules.sort_values(by="lift", ascending=False)

# looking for group type gname* & attack 
pd.set_option("display.max_rows", None)
# filter with gname in regex gname* i want to 'gname_Syrian Democratic Forces (SDF)' 

# filter sur les rules qui ont gname dans antecedant ou consequent
#filtered_rules.loc[filtered_rules['antecedents'].apply(lambda x: 'gname' in str(x)) & filtered_rules['consequents'].apply(lambda x: 'attack' in str(x))].head(1000)


filtered_rules_subset = filtered_rules.loc[
    filtered_rules['antecedents'].apply(lambda x: len(str(x).split()) == 1 and 'gname' in str(x)) & 
    filtered_rules['consequents'].apply(lambda x: 'attack' in str(x))
]

top_lift_rules = filtered_rules_subset.sort_values('lift', ascending=False) \
                                      .groupby('antecedents') \
                                      .head(5)

top_lift_rules.head(1000)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
74673,(gname_Al-Shabaab),"(target_military, attack_bombing/explosion, claimed, success)",0.025806,0.025605,0.005444,0.210938,8.238189,0.004783,1.234877,0.901889
474477,(gname_Al-Shabaab),"(target_military, attack_bombing/explosion, claimed, crit1, success, weapon_explosives)",0.025806,0.025605,0.005444,0.210938,8.238189,0.004783,1.234877,0.901889
474855,(gname_Al-Shabaab),"(target_military, crit2, claimed, attack_bombing/explosion, success, weapon_explosives)",0.025806,0.025605,0.005444,0.210938,8.238189,0.004783,1.234877,0.901889
230535,(gname_Al-Shabaab),"(target_military, attack_bombing/explosion, claimed, crit1, success)",0.025806,0.025605,0.005444,0.210938,8.238189,0.004783,1.234877,0.901889
231403,(gname_Al-Shabaab),"(target_military, attack_bombing/explosion, claimed, crit2, success)",0.025806,0.025605,0.005444,0.210938,8.238189,0.004783,1.234877,0.901889
744181,(gname_Taliban),"(crit2, claimed, success, crit1, attack_unknown, weapon_unknown, crit3)",0.192339,0.008669,0.008266,0.042977,4.957340,0.006599,1.035848,0.988383
534704,(gname_Taliban),"(crit2, claimed, crit1, success, attack_unknown, crit3)",0.192339,0.008669,0.008266,0.042977,4.957340,0.006599,1.035848,0.988383
538105,(gname_Taliban),"(crit2, claimed, crit1, attack_unknown, weapon_unknown, crit3)",0.192339,0.009073,0.008468,0.044025,4.852551,0.006723,1.036562,0.982990
289062,(gname_Taliban),"(crit2, claimed, crit1, attack_unknown, crit3)",0.192339,0.009073,0.008468,0.044025,4.852551,0.006723,1.036562,0.982990
548941,(gname_Taliban),"(claimed, crit1, success, attack_unknown, weapon_unknown, crit3)",0.192339,0.008871,0.008266,0.042977,4.844673,0.006560,1.035638,0.982575
